In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Make sure that caffe is on the python path:
#caffe_root = '../'  # this file is expected to be in {caffe_root}/examples
import sys
import os
#print caffe_root
#sys.path.insert(0, caffe_root + 'python')
sys.path.append('/home/young/caffe-master/python/')
sys.path.append('/usr/lib/python2.7/dist-packages/')

import caffe

#rootdir = os.getcwd()
#print rootdir

print 'Done'

Done


In [2]:
# use classicication to test the extraction

# read image data from binary file use numpy.fromstring() and read()
rootdir = '/home/young/Desktop/hashmodel'
rootdata = rootdir + '/Zmnist_hash/dataset/'
TEST_DATA_FILE = rootdata + 't10k-images-idx3-ubyte'
TEST_LABEL_FILE = rootdata + 't10k-labels-idx1-ubyte'
test_num = 10000
with open(TEST_DATA_FILE, 'rb') as f:
    f.read(16)
    test_raw_data = np.fromstring(f.read(test_num * 28*28), dtype = np.uint8)
    
with open(TEST_LABEL_FILE, 'rb') as f:
    f.read(8)
    test_labels = np.fromstring(f.read(test_num), dtype = np.uint8)
# manually scale data instead of using `caffe.io.Transformer`
test_caffe_in = test_raw_data.reshape(test_num, 1, 28, 28) * 0.00390625

net = caffe.Net('/home/young/Desktop/hashmodel/mnist-64/lenet_64.prototxt', \
                '/home/young/Desktop/hashmodel/mnist-64/lenet_hash64_iter_20000.caffemodel')
net.set_mode_gpu() # use gpu model

test_out = net.forward_all(data = test_caffe_in)
test_prob = test_out['prob']

tmp = 0
for i in xrange(test_num):
    if (test_prob[i].argmax() == test_labels[i]):
        tmp = tmp+1
print tmp*100.0/test_num

99.03


In [3]:
# read image data from binary file use numpy.fromstring() and read()
rootdata = rootdir + '/Zmnist_hash/dataset/'
TEST_DATA_FILE = rootdata + 't10k-images-idx3-ubyte'
TEST_LABEL_FILE = rootdata + 't10k-labels-idx1-ubyte'
test_num = 10000
with open(TEST_DATA_FILE, 'rb') as f:
    f.read(16)
    test_raw_data = np.fromstring(f.read(test_num * 28*28), dtype = np.uint8)
    
with open(TEST_LABEL_FILE, 'rb') as f:
    f.read(8)
    test_labels = np.fromstring(f.read(test_num), dtype = np.uint8)
# manually scale data instead of using `caffe.io.Transformer`
test_caffe_in = test_raw_data.reshape(test_num, 1, 28, 28)
print test_caffe_in.shape

TRAIN_DATA_FILE = rootdata + 'train-images-idx3-ubyte'
TRAIN_LABEL_FILE = rootdata + 'train-labels-idx1-ubyte'
train_num = 60000
with open(TRAIN_DATA_FILE, 'rb') as f:
    f.read(16) # skip the header
    train_raw_data = np.fromstring(f.read(train_num * 28*28), dtype=np.uint8)

with open(TRAIN_LABEL_FILE, 'rb') as f:
    f.read(8) # skip the header
    train_labels = np.fromstring(f.read(train_num), dtype=np.uint8)
# manually scale data instead of using `caffe.io.Transformer`
train_caffe_in = train_raw_data.reshape(train_num, 1, 28, 28)

print train_caffe_in.shape

(10000, 1, 28, 28)
(60000, 1, 28, 28)


In [5]:
from scipy.io import savemat
savemat('label.mat', {'test_label':test_labels, 'train_label':train_labels})

savemat('data.mat', {'test':test_caffe_in, 'train':train_caffe_in})

In [7]:
# iterate all the bits from 16~512
from scipy.io import savemat

rootdir = '/home/young/Desktop/hashmodel'

netfile = ['16', '24', '32', '48', '64', '128', '256', '512']
modelfile = ['lenet_hash16_iter_20000.caffemodel', 'lenet_hash24_iter_20000.caffemodel', \
             'lenet_hash32_iter_20000.caffemodel', 'lenet_hash48_iter_20000.caffemodel', \
             'lenet_hash64_iter_20000.caffemodel', 'lenet_hash128_iter_20000.caffemodel', \
             'lenet_hash256_iter_20000.caffemodel', 'lenet_feature512_iter_20000.caffemodel']

# read image data from binary file use numpy.fromstring() and read()
rootdata = rootdir + '/Zmnist_hash/dataset/'
TEST_DATA_FILE = rootdata + 't10k-images-idx3-ubyte'
TEST_LABEL_FILE = rootdata + 't10k-labels-idx1-ubyte'
test_num = 10000
with open(TEST_DATA_FILE, 'rb') as f:
    f.read(16)
    test_raw_data = np.fromstring(f.read(test_num * 28*28), dtype = np.uint8)
    
with open(TEST_LABEL_FILE, 'rb') as f:
    f.read(8)
    test_labels = np.fromstring(f.read(test_num), dtype = np.uint8)
# manually scale data instead of using `caffe.io.Transformer`
test_caffe_in = test_raw_data.reshape(test_num, 1, 28, 28) * 0.00390625

TRAIN_DATA_FILE = rootdata + 'train-images-idx3-ubyte'
TRAIN_LABEL_FILE = rootdata + 'train-labels-idx1-ubyte'
train_num = 60000
with open(TRAIN_DATA_FILE, 'rb') as f:
    f.read(16) # skip the header
    train_raw_data = np.fromstring(f.read(train_num * 28*28), dtype=np.uint8)

with open(TRAIN_LABEL_FILE, 'rb') as f:
    f.read(8) # skip the header
    train_labels = np.fromstring(f.read(train_num), dtype=np.uint8)
# manually scale data instead of using `caffe.io.Transformer`
train_caffe_in = train_raw_data.reshape(train_num, 1, 28, 28) * 0.00390625 



roothash = rootdir + '/Zmnist_hash/hash/'

for i in xrange(8):
    print 'processing %s bit hashing' % netfile[i]
    rootmodel = rootdir + '/mnist-' + netfile[i] + '/'
    net = caffe.Net(rootmodel + 'lenet_' + netfile[i] + '.prototxt', \
                    rootmodel + modelfile[i])
    net.set_mode_gpu() # use gpu model
    test_out = net.forward_all(data = test_caffe_in)
    train_out = net.forward_all(data = train_caffe_in)
    #hash = np.vstack((test_out.get('hash'+netfile[i]), train_out.get('hash'+netfile[i])))
    test_hash = test_out.get('hash'+netfile[i])
    train_hash = train_out.get('hash'+netfile[i])
    savemat(roothash+'hash'+netfile[i]+'.mat', {'testh':test_hash, 'trainh':train_hash})

#label = np.ndarray((test_num+train_num,), dtype = np.uint8)
#label[0:test_num] = test_labels
#label[test_num:test_num+train_num] = train_labels
savemat(roothash+'label.mat', {'test_label':test_labels, 'train_label':train_labels})

print 'Done'

processing 16 bit hashing
processing 24 bit hashing
processing 32 bit hashing
processing 48 bit hashing
processing 64 bit hashing
processing 128 bit hashing
processing 256 bit hashing
processing 512 bit hashing
Done
